In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import os
import argparse
import pickle as pkl
import time
from copy import deepcopy
import torch.optim.lr_scheduler as lr_scheduler
from tqdm import tqdm

In [2]:
import os
import pandas as pd
from torchvision.io import read_image
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from random import shuffle
import torch.nn.functional as F
from torchvision.transforms import v2
from torch import nn

c:\Users\nikos\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
c:\Users\nikos\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback

In [3]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None, end_maxpool = False):
        super(ResidualBlock, self).__init__()
        if(downsample is not None):
            self.conv1 = nn.Sequential(
                            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding='same'),
                            nn.BatchNorm2d(out_channels),
                            nn.ReLU(inplace=False),
                            nn.MaxPool2d(kernel_size=2, stride=2)
                            )  # Changed inplace to False
        else:
            self.conv1 = nn.Sequential(
                            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding='same'),
                            nn.BatchNorm2d(out_channels),
                            nn.ReLU(inplace=False)
                            )
        self.conv2 = nn.Sequential(
                        nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1),
                        nn.BatchNorm2d(out_channels),
                        nn.ReLU(inplace=False))  # Changed inplace to False
        self.downsample = downsample
        self.relu = nn.Hardtanh(min_val=0,max_val=1,inplace=False)  # Changed inplace to False
        self.out_channels = out_channels
        self.end_maxpool = end_maxpool

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.conv2(out)
        if self.downsample:
            residual = self.downsample(x)
        out = out + residual
        if self.end_maxpool:
            out = self.relu(out)
        else:
            out = self.relu(out)   # Use non-in-place ReLU
        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes = 2, in_channels = 5):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Sequential(
                        nn.Conv2d(in_channels, 64, kernel_size = 7, stride = 2, padding = 3),
                        nn.BatchNorm2d(64),
                        nn.ReLU(inplace=False))
        self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
        self.layer0 = self._make_layer(block, 64, layers[0], stride = 1)
        self.layer1 = self._make_layer(block, 128, layers[1], stride = 2)
        self.layer2 = self._make_layer(block, 256, layers[2], stride = 2)
        self.layer3 = self._make_layer(block, 512, layers[3], stride = 2, end_maxpool = True)
        self.avgpool = nn.MaxPool2d(7, stride=1)
        self.fc = nn.Linear(512, 256)
        self.fc2 = nn.Linear(256, num_classes)
        self.dropout = nn.Dropout(0.1)

    def _make_layer(self, block, planes, blocks, stride=1, end_maxpool = False):
        downsample = None
        if stride != 1 or self.inplanes != planes:

            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes, kernel_size=1, stride=1, padding='same'),
                nn.BatchNorm2d(planes),
                nn.ReLU(inplace=False),
                nn.MaxPool2d(kernel_size=2, stride=2)
            )
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes
        for i in range(1, blocks):
            if i == blocks-1 and end_maxpool:
                layers.append(block(self.inplanes, planes, end_maxpool = True))
            else:
                layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        x = F.hardtanh(x,min_val=0, max_val=1,inplace=False)
        # x = self.dropout(x)
        x = self.fc2(x)
        return x# F.hardtanh(x)

In [4]:
# model_resnet = ResNet(ResidualBlock, [5, 6, 6, 4], in_channels = 6, num_classes=101).to("cuda")

In [5]:
model_resnet = ResNet(ResidualBlock, [5, 6, 6, 4], in_channels = 6, num_classes=101)

In [6]:
model_resnet.load_state_dict(torch.load("best_resnet_nCaltech101_ReLU_ReLU1_ReLUmaxpool_EST__FC2_corrected_exp_mixed_L1_L2_Vol2.pt", weights_only=True))
model_resnet.eval().to("cuda")
print(model_resnet)

c:\Users\nikos\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


ResNet(
  (conv1): Sequential(
    (0): Conv2d(6, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer0): Sequential(
    (0): ResidualBlock(
      (conv1): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (conv2): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (relu): Hardtanh(min_val=0, max_val=1)
    )
    (1): ResidualBlock(
      (conv1): Sequential(
        (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
        (1): BatchN

In [7]:
temp = torch.rand((1,6,224,224))
model_resnet(temp.to("cuda"))

tensor([[-0.3461,  2.3041,  0.8275, -0.8413, -0.2564, -0.6802,  0.7623,  1.5938,
         -3.6353, -2.2463, -1.1614, -3.7241, -1.2510, -1.1890, -2.6656, -2.7113,
         -0.1521, -1.1356,  0.3712, -2.2424, -2.8217,  0.5626, -1.3454,  1.8509,
          0.8846,  2.1217,  2.9635, -1.7746, -3.9600, -3.5792,  4.2867,  0.7533,
         -1.2484, -4.5204, -2.0599,  0.6192, -1.8152, -2.1875, -0.8493, -1.0528,
          1.5820, -4.5909,  0.6464, -0.2655, -1.8212,  3.4811, -1.8866,  2.0174,
         -0.7050,  0.3872, -0.8142, -2.2276, -1.9676, -0.5916, -3.8016,  3.0546,
          1.5659, -1.9766, -1.8108, -1.0348, -0.2905,  2.9969, -0.6359, -1.5118,
          2.9575, -2.6695, -0.5463, -0.5480, -2.0595, -1.4532, -1.7102,  2.9516,
         -0.0385, -0.4666, -0.0113, -0.7729, -2.7480, -1.4833, -3.1092, -2.3030,
         -1.7359,  0.6253,  0.1460, -2.9708, -3.2043,  2.5897,  1.6775, -2.3769,
         -4.6142, -2.4093, -2.9785, -4.5807,  1.4476, -1.7224, -4.3998,  0.6697,
         -6.0469,  1.0178, -

In [8]:
from SNN.Check_ReLU1_ReLU import fuse_conv_and_bn
from SNN.Check_ReLU1_ReLU import SpikingConv2D
from SNN.Check_ReLU1_ReLU import MaxMinPool2D
from SNN.Check_ReLU1_ReLU import LayerSNN_ReLU1
from SNN.Check_ReLU1_ReLU import SpikingDense_positive_ReLU1, SpikingDense_pos_all, SpikingDense

In [9]:
class ResNet_ttfs(nn.Module):
    def __init__(self, model : ResNet, in_channels = 5):
        super(ResNet_ttfs, self).__init__()
        model.eval()
        robustness_params={
            'noise':0.0,
            'time_bits':0,
            'weight_bits': 0,
            'latency_quantiles':0.0
        }
        model.to('cuda')
        conv_fused = fuse_conv_and_bn(model.conv1[0], model.conv1[1], device = 'cuda')
        self.conv_first = SpikingConv2D(64, "temp1", device = 'cuda', padding=(3,3), stride=2, kernel_size=(7,7),robustness_params=robustness_params, kernels=conv_fused.weight.data, biases= conv_fused.bias.data)
        max_vect = torch.tensor([1]*in_channels)
        tmin, tmax, max_vect, scalar = self.conv_first.set_params(0.0,1.0,max_vect)
        self.tmin_post_pool = tmax
        self.pool = MaxMinPool2D(3, tmax.data,2,1).to("cuda")
        self.layer0SNN = LayerSNN_ReLU1(model.layer0, 64, 64, 5,device = 'cuda')
        tmax_prev = tmax
        tmin, tmax, max_vect, scalar = self.layer0SNN.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.layer1SNN = LayerSNN_ReLU1(model.layer1, 64, 128, 6,device = 'cuda')
        tmin, tmax, max_vect, scalar = self.layer1SNN.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.layer2SNN = LayerSNN_ReLU1(model.layer2, 128, 256, 6,device = 'cuda')
        tmin, tmax, max_vect, scalar = self.layer2SNN.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.layer3SNN = LayerSNN_ReLU1(model.layer3, 256, 512, 4,device = 'cuda',end_maxpool=True)
        tmin, tmax, max_vect, scalar = self.layer3SNN.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.pool2 = MaxMinPool2D(7, tmax.data,1,0).to("cuda")
        # temp = torch.ones((512,2,2))
        # max_vect = ((temp.T)*max_vect[:512]).T
        # max_vect = max_vect.view(-1)

        max_vect = (torch.ones([512, 1, 1]).to("cuda").T*max_vect).T.contiguous().view(512).to("cuda")

        weights = model.fc.weight.detach().clone()
        biases = model.fc.bias.detach().clone()
        self.layer_fc = SpikingDense_positive_ReLU1(256, 512,"test",robustness_params=robustness_params,device = 'cuda',weights=weights, biases=biases)
        
        tmin, tmax, max_vect, scalar = self.layer_fc.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.layer_fc2 = SpikingDense_pos_all(101,256, '',model.fc2.weight, model.fc2.bias,robustness_params=robustness_params,device = 'cuda')
        tmin, tmax, max_vect, scalar = self.layer_fc2.set_params(tmin, tmax, max_vect, in_scalar=scalar)
        self.tmin, self.tmax = tmin, tmax
        self.scalar = scalar
    
    def forward(self, tj):
        spike_sum, v_sqare_sum = 0,0.0

        x, spike, v  = self.conv_first(tj)

        spike_sum += spike
        v_sqare_sum +=v
        x, spike = self.pool(x)

        spike_sum += spike
        # print(x.shape)
        x = x
        x, spike1, v1 = self.layer0SNN(x)

        x, spike2, v2 = self.layer1SNN(x)

        x, spike3, v3 = self.layer2SNN(x)

        x, spike4, v4 = self.layer3SNN(x)

        # print(x[1].max(),x[1].min())
        x, spike5 = self.pool2(x)
        x = x.contiguous().view(x.size(0), -1)
        x, spike6, v5 = self.layer_fc(x)

        x, spike7, v6 = self.layer_fc2(x)

        spike_sum += spike1+spike2+spike3+spike4+spike5+spike6+spike7
        v_sqare_sum += v1+v2+v3+v4+v5+v6
        return x, spike_sum, v_sqare_sum

In [10]:
model_ttfs = ResNet_ttfs(model_resnet,6)

c:\D\time_to_spike\in_torch\SNN\Check_ReLU1_ReLU.py:280: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.t_max = torch.tensor(t_min + self.B_n*max_V+eps_V, dtype=torch.float64, requires_grad=False)
c:\D\time_to_spike\in_torch\SNN\Check_ReLU1_ReLU.py:311: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.t_max = torch.tensor(max(t_min + self.B_n*max_V+eps_V, minimal_t_max), dtype=torch.float64, requires_grad=False)
c:\D\time_to_spike\in_torch\SNN\Check_ReLU1_ReLU.py:279: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.t_min = torch.

In [11]:
temp = torch.rand((1,6,224,224))
temp_ttfs = 1 - temp
# print(model_ttfs.forward(temp_ttfs))
out, spike_sum, v_sqare_sum = model_ttfs.forward(temp_ttfs.to("cuda"))

print(spike_sum, v_sqare_sum)
print(((model_ttfs.tmax - out)*model_ttfs.scalar)[0,:101] - ((model_ttfs.tmax - out)*model_ttfs.scalar)[0,101:])

tensor(5130027, device='cuda:0') tensor(981.5477, device='cuda:0', grad_fn=<AddBackward0>)
tensor([-6.8941e-01,  2.1747e+00,  8.9856e-01, -6.6238e-01, -2.6113e-01,
        -6.7236e-01,  5.7631e-01,  1.3214e+00, -3.8275e+00, -2.3663e+00,
        -1.0033e+00, -4.1186e+00, -1.1306e+00, -1.0853e+00, -2.5061e+00,
        -2.2453e+00,  1.7796e-01, -1.1310e+00,  1.2183e-01, -2.1526e+00,
        -2.5688e+00,  5.8795e-01, -8.3951e-01,  2.3327e+00,  1.3701e+00,
         1.8599e+00,  3.0208e+00, -1.9559e+00, -3.5630e+00, -3.8874e+00,
         3.9763e+00,  6.0084e-01, -1.2923e+00, -4.4949e+00, -2.4304e+00,
         6.8774e-01, -1.8125e+00, -2.0458e+00, -9.4721e-01, -1.0482e+00,
         1.6774e+00, -4.8895e+00,  8.3369e-01, -1.5302e-01, -1.4008e+00,
         3.6150e+00, -2.2819e+00,  1.7984e+00, -3.6674e-01,  4.1539e-01,
        -3.7339e-01, -1.7456e+00, -1.7194e+00, -1.0029e+00, -4.1327e+00,
         2.5210e+00,  1.7227e+00, -1.9343e+00, -1.4690e+00, -7.7880e-01,
        -3.6757e-01,  2.5622e+00,

In [12]:
model_resnet(temp.to("cuda"))

tensor([[-6.9874e-01,  2.1758e+00,  9.0458e-01, -6.6580e-01, -2.6192e-01,
         -6.6109e-01,  5.6925e-01,  1.3217e+00, -3.8213e+00, -2.3737e+00,
         -1.0085e+00, -4.1239e+00, -1.1252e+00, -1.0906e+00, -2.4990e+00,
         -2.2449e+00,  1.8182e-01, -1.1431e+00,  1.1407e-01, -2.1461e+00,
         -2.5725e+00,  5.8729e-01, -8.3353e-01,  2.3449e+00,  1.3730e+00,
          1.8621e+00,  3.0236e+00, -1.9583e+00, -3.5541e+00, -3.8969e+00,
          3.9821e+00,  5.9304e-01, -1.3015e+00, -4.4933e+00, -2.4403e+00,
          6.8198e-01, -1.8190e+00, -2.0344e+00, -9.5969e-01, -1.0358e+00,
          1.6807e+00, -4.8931e+00,  8.3407e-01, -1.4981e-01, -1.3875e+00,
          3.6068e+00, -2.2860e+00,  1.7953e+00, -3.6570e-01,  4.2418e-01,
         -3.6467e-01, -1.7422e+00, -1.7207e+00, -1.0093e+00, -4.1346e+00,
          2.5189e+00,  1.7185e+00, -1.9313e+00, -1.4665e+00, -7.7554e-01,
         -3.7850e-01,  2.5448e+00, -4.7451e-01, -2.0581e+00,  2.5805e+00,
         -2.1988e+00, -7.9554e-01, -6.

In [13]:
from torchvision.transforms import v2
torch.manual_seed(19)

transforms = v2.Compose([
    # v2.RandomResizedCrop(size=(224, 224), antialias=True),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomRotation(60),
    v2.ToDtype(torch.float32)
])


class NCaltech101ImageDataset(Dataset):
    def __init__(self, img_dir_file, transform=None, target_transform=None):
        self.images = np.load(img_dir_file + '_x.npy')
        self.labels = np.load(img_dir_file + '_y.npy')
        self.transform = transform
        self.transform = transform
        self.stage = 0

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        # if self.transform:
        #     image = self.transform(image)
        # if self.target_transform:
        #     label = self.target_transform(label)
        label_temp = np.zeros((101,))
        label_temp[label] = 1
        if self.stage == 0:
            return self.transform(torch.tensor(image)), torch.tensor(label_temp)
        else:
            return torch.tensor(image), torch.tensor(label_temp)
    
    def set_stage(self, stage):
        self.stage = stage

In [14]:
from torch.utils.data import DataLoader
# training_data = NCarsImageDataset("./Datasety/nCars_train_EST_exp_", transform=transforms)
data = NCaltech101ImageDataset("./Datasety/Ncaltech101_EST_exp_corr", transform=transforms)


In [15]:
data.set_stage(1)
spike_acc = 0
MAE_sum = 0
MSE_sum = 0
TP = 0
for i in tqdm(range(len(data))):#10
    with torch.no_grad():
        with torch.cuda.amp.autocast():
            temp_data, temp_label = deepcopy(data[i])
            temp_ttfs = (1 - temp_data).unsqueeze(0).to("cuda")
            temp_data = temp_data.to("cuda")
            # print(temp_data.shape)
            # print(model_ttfs.forward(temp_ttfs))
            out, spike_sum, v_sqare_sum = model_ttfs.forward(temp_ttfs)
            spike_acc+=spike_sum
            output = ((model_ttfs.tmax - out)*model_ttfs.scalar)[0,:101] - ((model_ttfs.tmax - out)*model_ttfs.scalar)[0,101:]
            output_true = model_resnet(temp_data.unsqueeze(0))
            MAE = torch.sum((output-output_true).abs())/101.0
            MSE = torch.sqrt(torch.sum((output-output_true)*(output-output_true)))
            MAE_sum += MAE
            MSE_sum += MSE
            TP += (torch.argmax(temp_label)==torch.argmax(output)) 
            del temp_data, temp_ttfs
    if i%100==0:
        torch.cuda.empty_cache()
print(spike_acc/len(data))
print("Mean MAE:", MAE_sum/len(data))
print("Mean MSE:", MSE_sum/(len(data)*101.0))
print("accuracy:",float(TP)/len(data))

100%|██████████| 8709/8709 [1:03:53<00:00,  2.27it/s]

tensor(5576975.5000, device='cuda:0')
Mean MAE: tensor(0.0091, device='cuda:0')
Mean MSE: tensor(0.0011, device='cuda:0')
accuracy: 0.7269491330807211


In [16]:
print(TP)

tensor(6331, device='cuda:0')


In [17]:
i = 8000

with torch.no_grad():
    with torch.cuda.amp.autocast():
        temp_data, temp_label = deepcopy(data[i])
        temp_ttfs = (1 - temp_data).unsqueeze(0).to("cuda")
        temp_data = temp_data.to("cuda")
        # print(temp_data.shape)
        # print(model_ttfs.forward(temp_ttfs))
        out, spike_sum, v_sqare_sum = model_ttfs.forward(temp_ttfs)
        spike_acc+=spike_sum
        output = ((model_ttfs.tmax - out)*model_ttfs.scalar)[0,:101] - ((model_ttfs.tmax - out)*model_ttfs.scalar)[0,101:]
        output_true = model_resnet(temp_data.unsqueeze(0))
        MAE = torch.sum((output-output_true).abs())/101.0
        MSE = torch.sqrt(torch.sum((output-output_true)*(output-output_true)))
        MAE_sum += MAE
        MSE_sum += MSE
        TP += (torch.argmax(temp_label)==torch.argmax(output)) 
        print(torch.argmax(temp_label), torch.argmax(output))
        del temp_data, temp_ttfs

tensor(91) tensor(91, device='cuda:0')
